# 전처리 해결 과제 
### **1. 정제**
- 태그 제거 --> 완료
  - ex. \<p>\</p> 등
- 문자 제거 --> 완료
  - ex. \n, [1] 등
- 불용어 제거 
  - ex. '위키백과', '편집' 등
- 소제목별 내용 구분
- 표 문제 해결

### **2. 요약문 (목차 이전 텍스트)**
1. 좌표 문제 해결
2. p 태그 여러 개에 요약문 나오는 경우
  - 소제목별 내용 구분과 비슷한 맥락일 수 있음

---



# 준비
- import 
- data load

In [9]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# dev00을 df로 만든 데이터 사용
df = pd.read_pickle('/content/drive/MyDrive/data/Final PJT/df_00.pkl')
df.head(5)

,title,url,context,raw_html,qas
0,2016년_하계_올림픽_배구_남자_선수_명단,https://ko.wikipedia.org/wiki/2016년_하계_올림픽_배구_...,<!DOCTYPE html>\n<html>\n<head>\n<meta>\n<titl...,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...","[{'answer': {'text': '<td>332cm</td>', 'html_a..."
1,혈액_검사,https://ko.wikipedia.org/wiki/혈액_검사,<!DOCTYPE html>\n<html>\n<head>\n<meta>\n<titl...,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...","[{'answer': {'text': '<td>145</td>', 'html_ans..."
2,안토닌_드보르자크,https://ko.wikipedia.org/wiki/안토닌_드보르자크,<!DOCTYPE html>\n<html>\n<head>\n<meta>\n<titl...,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...","[{'answer': {'text': '사이프러스', 'html_answer_sta..."
3,조석_가속,https://ko.wikipedia.org/wiki/조석_가속,<!DOCTYPE html>\n<html>\n<head>\n<meta>\n<titl...,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...","[{'answer': {'text': '1695년', 'html_answer_sta..."
4,조영무,https://ko.wikipedia.org/wiki/조영무,<!DOCTYPE html>\n<html>\n<head>\n<meta>\n<titl...,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...","[{'answer': {'text': '<ul><li>할아버지 : 조순후, 증 참찬..."


# >> Korquad 함수
KorQuad 사이트에서 찾아온 함수 코드

In [ ]:
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys
import os
from bs4 import BeautifulSoup

In [ ]:
def normalize_answer(s):    
    def tag_clean(t):
        return BeautifulSoup(t).get_text()

    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text) 
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)   
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)      
        return text

    def white_space_fix(text):
        return ' '.join(text.split()).replace('\n','').replace('\t','').replace(' ','')

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(tag_clean(s)))))


In [ ]:
html_c = df['context'][0]
soup_c = BeautifulSoup(html_c, 'html.parser')
s = soup_c.select('p')
s

[<p><b>2016년 하계 올림픽 배구 남자 선수명단</b>은 2016년 8월6일부터 8월21일까지 <a>브라질</a> <a>리우데자네이루</a>에서 열리는 <a>제31회 하계 올림픽 남자 배구</a> 경기에 출전한 12개 남자 국가대표팀 선수 명단이다.
 </p>, <p>2016년 하계 올림픽에 출전한 멕시코 남자 배구 대표팀 명단이다.<sup><a>[1]</a></sup>
 </p>, <p>감독: 호르헤 아자이르 (Jorge Azair)
 </p>, <p>2016년 하계 올림픽에 출전한 미국 남자 배구 대표팀 명단이다.<sup><a>[2]</a></sup>
 </p>, <p>감독: 존 스피로(John Speraw)
 </p>, <p>2016년 하계 올림픽에 출전한 브라질 남자 배구 대표팀 명단이다.<sup><a>[3]</a></sup>
 </p>, <p>감독: 베르나르두 헤젠지 (Bernardo Rezende)
 </p>, <p>2016년 하계 올림픽에 출전한 이탈리아 남자 배구 대표팀 명단이다.<sup><a>[4]</a></sup>
 </p>, <p>감독: 잔로렌초 블렌지니(Gianlorenzo Blengini)
 </p>, <p>2016년 하계 올림픽에 출전한 캐나다 남자 배구 대표팀 명단이다.<sup><a>[5]</a></sup>
 </p>, <p>감독: 글렌 호그(Glenn Hoag)
 </p>, <p>2016년 하계 올림픽에 출전한 프랑스 남자 배구 대표팀 명단이다.<sup><a>[6]</a></sup>
 </p>, <p>감독: 로랑 틸리에 (Laurent Tillie)
 </p>, <p>2016년 하계 올림픽에 출전한 러시아 남자 배구 대표팀 명단이다.<sup><a>[7]</a></sup>
 </p>, <p>감독: 블라디미르 알레크노(Vladimir Alekno)
 </p>, <p>2016년 하계 올림픽에 출전한 아르헨티나 남자 배구 대표팀 명단이다.<sup><a>[8]</a></sup>
 </p>, <p>감독: 훌리오 벨라스코 (Julio V

In [ ]:
normalize_answer(html_c)

'2016년하계올림픽배구남자선수명단위키백과우리모두의백과사전2016년하계올림픽배구남자선수명단위키백과우리모두의백과사전둘러보기로가기검색하러가기2016년하계올림픽배구경기남자여자선수명단남자여자비치발리볼경기남자여자2016년하계올림픽배구남자선수명단은2016년8월6일부터8월21일까지브라질리우데자네이루에서열리는제31회하계올림픽남자배구경기에출전한12개남자국가대표팀선수명단이다목차1a조11멕시코12미국13브라질14이탈리아15캐나다16프랑스2b조21러시아22아르헨티나23이란24이집트25쿠바26폴란드3같이보기4각주a조편집멕시코편집2016년하계올림픽에출전한멕시코남자배구대표팀명단이다1감독호르헤아자이르jorgeazair번호이름출생일키몸무게스파이크높이가로막기높이2015–16년소속팀01\ufeff1다니엘바르가스danielvargas1986년9월1일197cm94kg340cm330cm라이시온로이무04\ufeff4곤잘로루이스gonzaloruiz1988년4월24일186cm87kg345cm325cmimssatn05\ufeff5헤수스란헬jesúsrangel리베로1980년9월20일190cm82kg337cm330cm티그레스uanl06\ufeff6헤수스알베르토페랄레스jesúsalbertoperales1993년12월22일197cm88kg328cm304cm티그레스uanl07\ufeff7호르헤키뇨네스jorgequiñones1981년11월13일186cm80kg330cm325cm비르투스가나후아토09\ufeff9카를로스게라carlosguerra주장1981년8월3일196cm95kg348cm335cm셰누아주네브10페드로란헬pedrorangel1988년9월16일192cm85kg340cm324cm티그레스uanl11호르헤바라하스jorgebarajas1991년5월7일188cm80kg320cm317cm코코테로스데콜리마13사무엘코르도바samuelcórdova1989년3월13일200cm89kg353cm335cm바하칼리포르니아14토마스아길레라tomásaguilera1988년11월15일202cm95kg350cm340cm치와와17네스토르오렐

# context 확인

In [4]:
df['context'][0]

'<!DOCTYPE html>\n<html>\n<head>\n<meta>\n<title>2016년 하계 올림픽 배구 남자 선수 명단 - 위키백과, 우리 모두의 백과사전</title>\n\n\n<link>\n\n<meta>\n<link>\n<meta>\n<meta>\n<meta>\n<meta>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n\n</head>\n<body>\n<div></div>\n<div></div>\n<div>\n<a></a>\n<div></div>\n<div>\n</div>\n<h1>2016년 하계 올림픽 배구 남자 선수 명단</h1>\n<div>\n<div>위키백과, 우리 모두의 백과사전.</div>\n<div></div>\n<div></div>\n<a>둘러보기로 가기</a>\n<a>검색하러 가기</a>\n<div><div><table>\n<tbody><tr>\n<th colspan="5"><a>2016년 하계<br/>올림픽 배구</a>\n</th></tr>\n<tr>\n<th colspan="5"><a><img></a>\n</th></tr>\n<tr>\n<td>경기\n</td>\n<td>\xa0\n</td>\n<td><a>남자</a>\n</td>\n<td>\xa0\n</td>\n<td><a>여자</a>\n</td></tr>\n<tr>\n<td>선수명단\n</td>\n<td>\xa0\n</td>\n<td><a>남자</a>\n</td>\n<td>\xa0\n</td>\n<td><a>여자</a>\n</td></tr>\n<tr>\n<th colspan="5">비치발리볼<br/><a><img></a>\n</th></tr>\n<tr>\n<td>경기\n</td>\n<td>\n</td>\n<td><a>남자</a>\n</td>\n<td>\n</td>\n<td><a>여자</a>\n</td></tr></tbody></table>\n<p><b>2016

In [5]:
print(df['context'][0])

<!DOCTYPE html>
<html>
<head>
<meta>
<title>2016년 하계 올림픽 배구 남자 선수 명단 - 위키백과, 우리 모두의 백과사전</title>


<link>

<meta>
<link>
<meta>
<meta>
<meta>
<meta>
<link>
<link>
<link>
<link>
<link>
<link>
<link>
<link>
<link>
<link>
<link>

</head>
<body>
<div></div>
<div></div>
<div>
<a></a>
<div></div>
<div>
</div>
<h1>2016년 하계 올림픽 배구 남자 선수 명단</h1>
<div>
<div>위키백과, 우리 모두의 백과사전.</div>
<div></div>
<div></div>
<a>둘러보기로 가기</a>
<a>검색하러 가기</a>
<div><div><table>
<tbody><tr>
<th colspan="5"><a>2016년 하계<br/>올림픽 배구</a>
</th></tr>
<tr>
<th colspan="5"><a><img></a>
</th></tr>
<tr>
<td>경기
</td>
<td> 
</td>
<td><a>남자</a>
</td>
<td> 
</td>
<td><a>여자</a>
</td></tr>
<tr>
<td>선수명단
</td>
<td> 
</td>
<td><a>남자</a>
</td>
<td> 
</td>
<td><a>여자</a>
</td></tr>
<tr>
<th colspan="5">비치발리볼<br/><a><img></a>
</th></tr>
<tr>
<td>경기
</td>
<td>
</td>
<td><a>남자</a>
</td>
<td>
</td>
<td><a>여자</a>
</td></tr></tbody></table>
<p><b>2016년 하계 올림픽 배구 남자 선수명단</b>은 2016년 8월6일부터 8월21일까지 <a>브라질</a> <a>리우데자네이루</a>에서 열리는 <a>제31회 하계 올림픽 남자 배구<

# 텍스트 추출

## raw_html 이용

In [6]:
html = df['raw_html'][0]
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# soup

In [ ]:
soup.select('p > b')

[<b>2016년 하계 올림픽 배구 남자 선수명단</b>]

In [ ]:
soup.select('p > a')

[<a href="/wiki/%EB%B8%8C%EB%9D%BC%EC%A7%88" title="브라질">브라질</a>,
 <a href="/wiki/%EB%A6%AC%EC%9A%B0%EB%8D%B0%EC%9E%90%EB%84%A4%EC%9D%B4%EB%A3%A8" title="리우데자네이루">리우데자네이루</a>,
 <a href="/wiki/2016%EB%85%84_%ED%95%98%EA%B3%84_%EC%98%AC%EB%A6%BC%ED%94%BD_%EB%82%A8%EC%9E%90_%EB%B0%B0%EA%B5%AC" title="2016년 하계 올림픽 남자 배구">제31회 하계 올림픽 남자 배구</a>]

In [ ]:
soup.select('p')[0]

<p><b>2016년 하계 올림픽 배구 남자 선수명단</b>은 2016년 8월6일부터 8월21일까지 <a href="/wiki/%EB%B8%8C%EB%9D%BC%EC%A7%88" title="브라질">브라질</a> <a href="/wiki/%EB%A6%AC%EC%9A%B0%EB%8D%B0%EC%9E%90%EB%84%A4%EC%9D%B4%EB%A3%A8" title="리우데자네이루">리우데자네이루</a>에서 열리는 <a href="/wiki/2016%EB%85%84_%ED%95%98%EA%B3%84_%EC%98%AC%EB%A6%BC%ED%94%BD_%EB%82%A8%EC%9E%90_%EB%B0%B0%EA%B5%AC" title="2016년 하계 올림픽 남자 배구">제31회 하계 올림픽 남자 배구</a> 경기에 출전한 12개 남자 국가대표팀 선수 명단이다.
</p>

In [ ]:
html1 = df['raw_html'][1]
soup1 = BeautifulSoup(html1, 'html.parser')

In [ ]:
soup1.select('p')[0]

<p><b>혈액 검사</b>(血液檢査, <span style="font-size: smaller;"><a href="/wiki/%EC%98%81%EC%96%B4" title="영어">영어</a>: </span><span lang="en">blood test</span>)는 <a href="/wiki/%ED%98%88%EC%95%A1" title="혈액">혈액</a>에서 의학적 정보를 얻기 위한 검사이다. 혈액은 주로 팔에 있는 <a href="/wiki/%EC%A0%95%EB%A7%A5" title="정맥">정맥</a>이나 손가락 끝에서 <a class="new" href="/w/index.php?title=%ED%94%BC%EB%B6%80_%EB%B0%91_%EB%B0%94%EB%8A%98&amp;action=edit&amp;redlink=1" title="피부 밑 바늘 (없는 문서)">바늘</a>을 이용하여 채취한다. 특정한 혈액 구성 성분에 대한 여러 가지 검사는 하나의 무리로 묶기도 한다. 혈액 검사는 <a href="/wiki/%EC%9D%98%EB%A3%8C" title="의료">의료</a> 행위 중 <a href="/wiki/%EC%A7%88%EB%B3%91" title="질병">질병</a> 유무, <a href="/wiki/%EB%AC%B4%EA%B8%B0%EC%A7%88" title="무기질">무기질</a> 영양 상태, <a href="/wiki/%EC%9D%98%EC%95%BD%ED%92%88" title="의약품">의약품</a>의 효과, <a class="mw-redirect" href="/wiki/%EA%B8%B0%EA%B4%80_(%EC%83%9D%EB%AC%BC%ED%95%99)" title="기관 (생물학)">장기</a> 기능 등 <a href="/wiki/%EC%83%9D%EB%A6%AC%ED%95%99" title="생리학">생리적</a> 및 <a href="/wiki/%EC%83%9D%ED%99%94%ED%95%99" title

## context 이용

In [ ]:
html_c = df['context'][0]
soup_c = BeautifulSoup(html_c, 'html.parser')

In [ ]:
soup_c.select('p')[0]

<p><b>2016년 하계 올림픽 배구 남자 선수명단</b>은 2016년 8월6일부터 8월21일까지 <a>브라질</a> <a>리우데자네이루</a>에서 열리는 <a>제31회 하계 올림픽 남자 배구</a> 경기에 출전한 12개 남자 국가대표팀 선수 명단이다.
</p>

## 좌표 문제 해결 --> 아직 미완

In [ ]:
df['context'][17]

'<!DOCTYPE html>\n<html>\n<head>\n<meta>\n<title>영파여자중학교 - 위키백과, 우리 모두의 백과사전</title>\n\n\n<link>\n\n<meta>\n<link>\n<meta>\n<meta>\n<meta>\n<meta>\n<meta>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n<link>\n\n</head>\n<body>\n<div></div>\n<div></div>\n<div>\n<a></a>\n<div></div>\n<div>\n</div>\n<h1>영파여자중학교</h1>\n<div>\n<div>위키백과, 우리 모두의 백과사전.</div>\n<div></div>\n<div></div>\n<a>둘러보기로 가기</a>\n<a>검색하러 가기</a>\n<div><div><p><span><span><a>좌표</a>: <span><a><span><span><span>북위 37° 31′ 59″</span> <span>동경 127° 07′ 14″</span></span></span><span>\ufeff / \ufeff</span><span><span>북위 37.5331233°  동경 127.1204382° </span><span>\ufeff / <span>37.5331233; 127.1204382</span></span></span></a></span></span></span>\n</p>\n<table>\n<tbody><tr>\n<td colspan="2">\n<div><div></div><span><big><big><b>영파여자중학교</b></big></big><br/><big>英坡女子中學校</big><br/><big>Youngpa Girls\' Middle School</big></span></div>\n</td></tr>\n<tr>\n<td colspan="2"><a><img></a>\n</td></tr>\n<tr>\n<

In [ ]:
df['raw_html'][17]

'<!DOCTYPE html>\n<html class="client-nojs" dir="ltr" lang="ko">\n<head>\n<meta charset="utf-8"/>\n<title>영파여자중학교 - 위키백과, 우리 모두의 백과사전</title>\n<script>document.documentElement.className=document.documentElement.className.replace(/(^|\\s)client-nojs(\\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"영파여자중학교","wgTitle":"영파여자중학교","wgCurRevisionId":23982651,"wgRevisionId":23982651,"wgArticleId":1141373,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["위키데이터에 없는 좌표","서울 송파구의 중학교","1967년 개교","대한민국의 여자 중학교"],"wgBreakFrames":!1,"wgPageContentLanguage":"ko","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"ko","wgMonthNames":["","1월","2월","3월","4월","5월","6월","7월","8월","9월","10월","11월","12월"],"wgMonthNamesShort":["","1","2","3","4","5","6","7","8","9","10","11","12"],"wgRelevantPageName":"영파

In [ ]:
html = df['context'][995]
soup = BeautifulSoup(html, 'html.parser')
text_raw = soup.select('div > p')[0]

print(soup.select('div > p')[0].text)
# text_refine(text_raw)

오쿠리가나（일본어: .mw-parser-output ruby>rt,.mw-parser-output ruby>rtc{font-feature-settings:"ruby"1}.mw-parser-output ruby.large{font-size:250%}.mw-parser-output ruby.large>rt,.mw-parser-output ruby.large>rtc{font-size:.3em}送 ()りがな, 送 (おく)り仮 (が)名 (な))는 일본어를 한자와 가나를 섞어써서 표기할 때, 한자로 표기한 와고(일본어: 和語, 倭語 (わご)), 또는 야마토코토바(일본어: 大和言葉)를 쉽게 읽게 하기 위해 한자 뒤에 덧붙는 가나를 뜻한다. 이는 한 단어 내에서 한자를 쉽게 읽게 하기 위해 붙은 가나에 해당하고, 단어 자체가 가나만으로 이루어진 조사 같은 경우는 오쿠리가나라고 부르지 않는다. 또한, 오쿠리가나는 한자가 훈독으로 읽혔을 때만 사용되며, 음독으로 읽힐 때는 사용되지 않는다.



In [ ]:
html = df['raw_html'][17]
soup = BeautifulSoup(html, 'html.parser')
text_raw = str(soup.select('div > p')[0])

print(soup.select('div > p')[0])
print(soup.select('div > p')[1])
text_refine(text_raw)

<p><span style="font-size: small;"><span id="coordinates"><a href="/wiki/%EC%A7%80%EB%A6%AC_%EC%A2%8C%ED%91%9C%EA%B3%84" title="지리 좌표계">좌표</a>: <span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?language=ko&amp;pagename=%EC%98%81%ED%8C%8C%EC%97%AC%EC%9E%90%EC%A4%91%ED%95%99%EA%B5%90&amp;params=37.5331233_N_127.1204382_E_"><span class="geo-nondefault"><span class="geo-dms" title="이 위치에 대한 지도와 항공사진 및 기타 정보를 보여줍니다"><span class="latitude">북위 37° 31′ 59″</span> <span class="longitude">동경 127° 07′ 14″</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="geo-dec" title="이 위치에 대한 지도와 항공사진 및 기타 정보를 보여줍니다">북위 37.5331233°  동경 127.1204382° </span><span style="display:none">﻿ / <span class="geo">37.5331233; 127.1204382</span></span></span></a></span></span></span>
</p>
<p><b>영파여자중학교</b>(英坡女子中學校)는 <a href="/wiki/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C" title="서울특별시">서울특별시</a> <a href="/w

NameError: ignored

In [ ]:
# soup = BeautifulSoup(df['raw_html'][17], 'html.parser')
# soup.select('div')

# 텍스트 정제 함수 생성

In [ ]:
# text 정제 함수 1

# def text_refine1(text_raw):
#   text_raw = text_raw.replace('\n','')
#   text = ''
#   status = 0
#   # <style> 태그 내용 모두 삭제
#   if '<style>' in text_raw:
#     idx_s = text_raw.find('<style>')
#     idx_f = text_raw.find('</style>')
#     text_raw[idx_s:idx_f]

#     deltxt = str(text_raw[idx_s:idx_f+8])
#     text_raw  = text_raw.replace(deltxt,'')
#   ###
#   for t in text_raw:
#     if t == '<': status=1
#     if status==0: text = text+t
#     elif t == '>': status=0

#   return(text)

In [10]:
# text 정제 함수 : 최종

def text_refine(text_raw): # text_raw는 str 형태

  # 필요없는 문자 제거
  text = re.sub('<style.*?>.*?</style>','',text_raw,re.I|re.S)
  text = re.sub('\n|\xa0|\ufeff','',text)
  text = re.sub('[[]+\d+[]]','',text)
  text = re.sub('[[주 ]+\d+[]]','',text)

  # 태그 제거
  text1 = ''
  for t in text:
    if t == '<': status=1
    if status==0: text1 = text1+t
    elif t == '>': status=0

  # 빈 괄호 제거 및 공백 제거
  if ' (' in text1:
    text1 = text1.replace(' (','(')
  if '（' in text1:
    text1 = text1.replace('（','(')
  if '()' in text1:
    text1 = text1.replace('()','')
  if '[*]' in text1:
    text1 = text1.replace('[*]','')


  return(text1)

In [11]:
# 확인
soup = BeautifulSoup(df['context'][995], 'html.parser')
temp = soup.select('div > p')[0]
temp = str(temp)
text_refine(temp)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Possible nested set at position 1
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Possible nested set at position 1
  if __name__ == '__main__':


'오쿠리가나(일본어: 送りがな, 送(おく)り仮(が)名(な))는 일본어를 한자와 가나를 섞어써서 표기할 때, 한자로 표기한 와고(일본어: 和語, 倭語(わご)), 또는 야마토코토바(일본어: 大和言葉)를 쉽게 읽게 하기 위해 한자 뒤에 덧붙는 가나를 뜻한다. 이는 한 단어 내에서 한자를 쉽게 읽게 하기 위해 붙은 가나에 해당하고, 단어 자체가 가나만으로 이루어진 조사 같은 경우는 오쿠리가나라고 부르지 않는다. 또한, 오쿠리가나는 한자가 훈독으로 읽혔을 때만 사용되며, 음독으로 읽힐 때는 사용되지 않는다.'

# 전체 데이터 - 첫 글 추출



In [12]:
len(df['raw_html']) # 총 1000개 문서

1000

In [13]:
df2 = df.copy()

In [ ]:
# 변경 후 -- text_refine 사용

text_li = []
for i in range(len(df['context'])):
  html = df['context'][i]
  soup = BeautifulSoup(html, 'html.parser')

  try:
    text_raw = str(soup.select('div > p')[0])
    text = text_refine(text_raw)

    # if text[:3] == '좌표:':
    #   text_raw = str(soup.select('div > p')[1])
    #   text = text_refine(text_raw)

  except:
    text = 'No text'
  text_li.append(text)

print(i, 'done')
# text_li

999 done


In [ ]:
for i in range(len(check)):
  print(check[i])
  print(check2[i])
  print()

(17, '좌표: 북위 37° 31′ 59″ 동경 127° 07′ 14″ / 북위 37.5331233°  동경 127.1204382°  / 37.5331233; 127.1204382')
(17, '영파여자중학교(英坡女子中學校)는 서울특별시 송파구 풍납동에 있는 사립 중학교이다.')

(29, '좌표: 북위 36° 29′ 52″ 동경 127° 15′ 57″ / 북위 36.497641°  동경 127.265806°  / 36.497641; 127.265806')
(29, '문화체육관광부(文化體育觀光部, Ministry of Culture, Sports and Tourism, 약칭: 문체부, MCST)는 문화·예술·영상·광고·출판·간행물·체육·관광, 국정에 대한 홍보 및 정부발표에 관한 사무를 관장하는 대한민국의 중앙행정기관이다. 2008년 2월 29일 문화관광부와 국정홍보처를 통합하여 발족하였으며, 세종특별자치시 갈매로 388 정부세종청사 15동에 위치하고 있다. 장관은 국무위원으로, 차관은 정무직공무원으로 보한다.')

(30, '좌표: 북위 35° 10′ 41″ 동경 129° 04′ 37″ / 북위 35.178192°  동경 129.076946°  / 35.178192; 129.076946')
(30, '부산지방고용노동청(釜山地方雇用勞動廳)은 대한민국 고용노동부의 소속기관이다. 2010년 7월 5일 발족하였으며, 부산광역시 연제구 연제로 36에 위치하고 있다. 청장은 고위공무원단 나등급에 속하는 일반직공무원으로 보한다.')

(55, '좌표: 북위 37° 06′ 14″ 동경 129° 01′ 15″ / 북위 37.103791°  동경 129.020729°  / 37.103791; 129.020729')
(55, '장성여자중학교(長省女子中學校)는 강원도 태백시 장성동에 있는 공립 중학교이다.')

(78, '좌표:  06h 45m 08.9173s, −16° 42′ 58.017″')
(78, "시리우스(Sirius)는 밤하늘에서 가장 밝은 별로, 천랑성(天狼星), 

In [ ]:
df2['text'] = text_li
df2

,title,url,context,raw_html,qas,text
0,2016년_하계_올림픽_배구_남자_선수_명단,https://ko.wikipedia.org/wiki/2016년_하계_올림픽_배구_...,<!DOCTYPE html>\n<html>\n<head>\n<meta>\n<titl...,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...","[{'answer': {'text': '<td>332cm</td>', 'html_a...",2016년 하계 올림픽 배구 남자 선수명단은 2016년 8월6일부터 8월21일까지 ...
1,혈액_검사,https://ko.wikipedia.org/wiki/혈액_검사,<!DOCTYPE html>\n<html>\n<head>\n<meta>\n<titl...,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...","[{'answer': {'text': '<td>145</td>', 'html_ans...","혈액 검사(血液檢査, 영어: blood test)는 혈액에서 의학적 정보를 얻기 위..."
2,안토닌_드보르자크,https://ko.wikipedia.org/wiki/안토닌_드보르자크,<!DOCTYPE html>\n<html>\n<head>\n<meta>\n<titl...,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...","[{'answer': {'text': '사이프러스', 'html_answer_sta...","안토닌 레오폴트 드보르자크(체코어: Antonín Leopold Dvořák, 18..."
3,조석_가속,https://ko.wikipedia.org/wiki/조석_가속,<!DOCTYPE html>\n<html>\n<head>\n<meta>\n<titl...,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...","[{'answer': {'text': '1695년', 'html_answer_sta...","조석 가속(潮汐 加速, 영어: Tidal acceleration)은 달과 같은 위성..."
4,조영무,https://ko.wikipedia.org/wiki/조영무,<!DOCTYPE html>\n<html>\n<head>\n<meta>\n<titl...,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...","[{'answer': {'text': '<ul><li>할아버지 : 조순후, 증 참찬...","조영무(趙英茂, 1338년 ~ 1414년 7월 28일)는 고려 말 조선 초의 무신이..."
...,...,...,...,...,...,...
995,오쿠리가나,https://ko.wikipedia.org/wiki/오쿠리가나,<!DOCTYPE html>\n<html>\n<head>\n<meta>\n<titl...,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...","[{'answer': {'text': '오독할 가능성을 낮추는 것', 'html_a...","오쿠리가나(일본어: 送 りがな, 送(おく)り仮(が)名(な))는 일본어를 한자와 가나..."
996,2015년_하계_유니버시아드,https://ko.wikipedia.org/wiki/2015년_하계_유니버시아드,<!DOCTYPE html>\n<html>\n<head>\n<meta>\n<titl...,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...","[{'answer': {'text': '서울', 'html_answer_start'...",제28회 하계 유니버시아드는 2015년 7월 3일부터 7월 14일까지 대한민국 광주...
997,로마_가톨릭교회,https://ko.wikipedia.org/wiki/로마_가톨릭교회,<!DOCTYPE html>\n<html>\n<head>\n<meta>\n<titl...,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...",[{'answer': {'text': '<p>가장 많이 알려진 대표적인 전례 양식은...,"로마 가톨릭교회(라틴어: Ecclesia Catholica Romana, 영어: R..."
998,토마스_헤슬러,https://ko.wikipedia.org/wiki/토마스_헤슬러,<!DOCTYPE html>\n<html>\n<head>\n<meta>\n<titl...,"<!DOCTYPE html>\n<html class=""client-nojs"" dir...","[{'answer': {'text': '4골', 'html_answer_start'...","토마스 위르겐 ""이케"" 헤슬러(Thomas Jürgen ""Icke"" Häßler, ..."


### 확인

In [ ]:
df2['url'][19]

'https://ko.wikipedia.org/wiki/교토세이카다이마에_역'

In [ ]:
soup_t.select('div.mw-content-ltr')[0].select('p')

[<p><span style="font-size: small;"><span id="coordinates"><a href="/wiki/%EC%A7%80%EB%A6%AC_%EC%A2%8C%ED%91%9C%EA%B3%84" title="지리 좌표계">좌표</a>: <span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?language=ko&amp;pagename=%EC%98%81%ED%8C%8C%EC%97%AC%EC%9E%90%EC%A4%91%ED%95%99%EA%B5%90&amp;params=37.5331233_N_127.1204382_E_"><span class="geo-nondefault"><span class="geo-dms" title="이 위치에 대한 지도와 항공사진 및 기타 정보를 보여줍니다"><span class="latitude">북위 37° 31′ 59″</span> <span class="longitude">동경 127° 07′ 14″</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="geo-dec" title="이 위치에 대한 지도와 항공사진 및 기타 정보를 보여줍니다">북위 37.5331233°  동경 127.1204382° </span><span style="display:none">﻿ / <span class="geo">37.5331233; 127.1204382</span></span></span></a></span></span></span>
 </p>,
 <p><b>영파여자중학교</b>(英坡女子中學校)는 <a href="/wiki/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C" title="서울특별시">서울특별시</a> <a href

In [ ]:
html_t = df['raw_html'][17]
soup_t = BeautifulSoup(html_t, 'html.parser')
temp = soup_t.select('div > p')

cont_t = df['context'][17]
soup_ct = BeautifulSoup(cont_t, 'html.parser')
temp_ct = soup_ct.select('div > p')

print(len(temp))
print(temp,'\n')
print(temp[0])
print(temp[0].text)
print('--------------------------------------------------------')
print(len(temp_ct))
print(temp_ct,'\n')
print(temp_ct[0])
print(temp_ct[0].text)

2
[<p><span style="font-size: small;"><span id="coordinates"><a href="/wiki/%EC%A7%80%EB%A6%AC_%EC%A2%8C%ED%91%9C%EA%B3%84" title="지리 좌표계">좌표</a>: <span class="plainlinks nourlexpansion"><a class="external text" href="//tools.wmflabs.org/geohack/geohack.php?language=ko&amp;pagename=%EC%98%81%ED%8C%8C%EC%97%AC%EC%9E%90%EC%A4%91%ED%95%99%EA%B5%90&amp;params=37.5331233_N_127.1204382_E_"><span class="geo-nondefault"><span class="geo-dms" title="이 위치에 대한 지도와 항공사진 및 기타 정보를 보여줍니다"><span class="latitude">북위 37° 31′ 59″</span> <span class="longitude">동경 127° 07′ 14″</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="geo-dec" title="이 위치에 대한 지도와 항공사진 및 기타 정보를 보여줍니다">북위 37.5331233°  동경 127.1204382° </span><span style="display:none">﻿ / <span class="geo">37.5331233; 127.1204382</span></span></span></a></span></span></span>
</p>, <p><b>영파여자중학교</b>(英坡女子中學校)는 <a href="/wiki/%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C" title="서울특별시">서울특별시</a> <a href

In [ ]:
for i in range(10,70):
  print(i, df2['title'][i],'//', df2['text'][i])
  print()

10 민진원 // 민진원(閔鎭遠, 1664년 ~ 1736년)은 조선 후기의 척신으로 민유중의 아들이며, 민진후의 남동생, 인현왕후의 작은오빠이다. 1691년 과거 급제 후, 인현왕후의 폐출로 빛을 보지 못하다가 1694년 갑술환국 이후 복귀하여 사복시정, 사헌부집의 등을 지냈고 노론(老論)에 참여하였다. 이후 전라도관찰사 등을 거쳐 좌의정을 역임했다.

11 미사 // 미사(라틴어: Missa)는 가톨릭에서 하느님을 찬양하는 대표적인 종교 의식이며, 특히 로마 가톨릭교회의 일곱 가지 성사 가운데 하나인 성체성사가 중심을 이루는 라틴 전례 양식이 일반적인 미사의 개념이다. 미사 전례의 집전은 로마 가톨릭교회에서 유효하게 서품받은 사제만이 할 수 있으며 부제, 복사 등과 함께 평신도는 고유 직분 및 역할로서 이에 참여한다.

12 2014년_FIFA_월드컵_F조 // 2014년 FIFA 월드컵 F조는 아르헨티나, 보스니아 헤르체고비나, 이란, 그리고 나이지리아로 구성되었다. F조 일정은 2014년 6월 15일에 시작해 6월 25일에 종료되었다.

13 베르너_하이젠베르크 // 베르너 카를 하이젠베르크(독일어: Werner Karl Heisenberg IPA: [ˈvɛʁnɐ kaʁl ˈhaɪ̯zn̩bɛʁk], 1901년 12월 5일~1976년 2월 1일)는 독일의 물리학자이다. 불확정성 원리로 유명하며, 행렬역학과 불확정성 원리를 발견하여 20세기 초 양자역학의 발전에 절대적인 공헌을 했다. 그는 1932년에 "양자역학을 창시한" 공로로 노벨 물리학상을 수상했다. 하이젠베르크는 막스 보른과 파스쿠알 요르단과 함께 양자역학의 행렬적 형식을 1925년 발표했다. 1927년에 하이젠베르크는 그의 가장 유명한 과학적 업적인 불확정성 원리를 출판했다. 또한, 그는 난류의 유체역학, 원자핵, 강자성, 우주선, 소립자의 연구에 지대한 공헌을 했다.

14 유옥우 // 유옥우(劉沃祐, 1914년 12월 4일 ~ 1984년 3월 15일)는 대한민국의 제3·4·5·8·11대 국회의원

In [ ]:
df['url'][2]

'https://ko.wikipedia.org/wiki/안토닌_드보르자크'

In [ ]:

soup_t = BeautifulSoup(df['context'][995], 'html.parser')

print(soup_t)
# print(str(soup_t.select('p')[0]))
# print(str(soup_t.select('p')[1]))
# print(str(soup_t.select('p')[2]))
# print(str(soup_t.select('p')[3]))
# print(str(soup_t.select('p')[4]))

<!DOCTYPE html>

<html>
<head>
<meta/>
<title>오쿠리가나 - 위키백과, 우리 모두의 백과사전</title>
<link/>
<meta/>
<link/>
<meta/>
<meta/>
<meta/>
<meta/>
<link/>
<link/>
<link/>
<link/>
<link/>
<link/>
<link/>
<link/>
<link/>
<link/>
<link/>
</head>
<body>
<div></div>
<div></div>
<div>
<a></a>
<div></div>
<div>
</div>
<h1>오쿠리가나</h1>
<div>
<div>위키백과, 우리 모두의 백과사전.</div>
<div></div>
<div></div>
<a>둘러보기로 가기</a>
<a>검색하러 가기</a>
<div><div><div><div><a><img/></a><br/>
<b><a>일본어 표기법</a></b></div><div>
<p><b><a>한자(칸지)</a></b>
</p>
<ul><li></li></ul>
<p><b><a>가나</a></b>
</p>
<ul><li><a>히라가나</a>
<ul><li><a>헨타이가나</a></li></ul></li>
<li><a>가타카나</a></li>
<li><a>만요가나</a></li></ul>
<p>사용법
</p>
<ul><li><a>후리가나</a></li>
<li><a>오쿠리가나</a></li></ul>
<p><b><a>로마자 표기법(로마지)</a></b>
</p>
<ul><li><a>헵번식</a></li>
<li><a>훈령식</a> (<a>ISO</a>)</li>
<li><a>일본식</a> (<a>전자법</a>)</li></ul>
<div><span><a><span>v</span></a> <span>•</span> <a><span>d</span></a> <span>•</span> <a><span>e</span></a> <span>•</span> <a><span>h</span></a></span>

## html

In [ ]:
html_t = df['raw_html'][995]
soup_t = BeautifulSoup(html_t, 'html.parser')
html_c = df['context'][995]
soup_c = BeautifulSoup(html_c, 'html.parser')

divLen = len(soup_t.select('div.mw-parser-output > div')[0].select('p'))
soup_t.select('div.mw-parser-output')[0].select('p')[divLen]


temp = str(soup_c.select('div > p')[0])
print(soup_c.select('div > p')[0])
print()
# .select('p')[divLen]
text_refine(str(soup_c.select('div > p')[0]))

<p><b>오쿠리가나</b>（<span><a>일본어</a>: </span><span><style>.mw-parser-output ruby>rt,.mw-parser-output ruby>rtc{font-feature-settings:"ruby"1}.mw-parser-output ruby.large{font-size:250%}.mw-parser-output ruby.large>rt,.mw-parser-output ruby.large>rtc{font-size:.3em}</style><span><ruby><rb>送</rb><rp> (</rp><rt></rt><rp>)</rp></ruby></span>りがな, <link/><span><ruby><rb>送</rb><rp> (</rp><rt>おく</rt><rp>)</rp></ruby></span>り<link/><span><ruby><rb>仮</rb><rp> (</rp><rt>が</rt><rp>)</rp></ruby></span><link/><span><ruby><rb>名</rb><rp> (</rp><rt>な</rt><rp>)</rp></ruby></span></span>)는 <a>일본어</a>를 <a>한자</a>와 <a>가나</a>를 섞어써서 표기할 때, 한자로 표기한 와고(<span><a>일본어</a>: </span><span>和語</span>, <link/><span><ruby><rb>倭語</rb><rp> (</rp><rt>わご</rt><rp>)</rp></ruby></span>), 또는 야마토코토바(<span><a>일본어</a>: </span><span>大和言葉</span>)를 쉽게 읽게 하기 위해 한자 뒤에 덧붙는 <a>가나</a>를 뜻한다. 이는 한 <a>단어</a> 내에서 한자를 쉽게 읽게 하기 위해 붙은 가나에 해당하고, 단어 자체가 가나만으로 이루어진 조사 같은 경우는 오쿠리가나라고 부르지 않는다. 또한, 오쿠리가나는 한자가 <a>훈독</a>으로 읽혔을 때만 사용되며, 음독으로 읽힐 때는 사용되지 않는다.
<

'오쿠리가나（일본어: .mw-parser-output ruby>rt,.mw-parser-output ruby>rtc{font-feature-settings:"ruby"1}.mw-parser-output ruby.large{font-size:250%}.mw-parser-output ruby.large>rt,.mw-parser-output ruby.large>rtc{font-size:.3em}送 ()りがな, 送 (おく)り仮 (が)名 (な))는 일본어를 한자와 가나를 섞어써서 표기할 때, 한자로 표기한 와고(일본어: 和語, 倭語 (わご)), 또는 야마토코토바(일본어: 大和言葉)를 쉽게 읽게 하기 위해 한자 뒤에 덧붙는 가나를 뜻한다. 이는 한 단어 내에서 한자를 쉽게 읽게 하기 위해 붙은 가나에 해당하고, 단어 자체가 가나만으로 이루어진 조사 같은 경우는 오쿠리가나라고 부르지 않는다. 또한, 오쿠리가나는 한자가 훈독으로 읽혔을 때만 사용되며, 음독으로 읽힐 때는 사용되지 않는다.'

In [ ]:
html_c = df['context'][0]
soup_c = BeautifulSoup(html_c, 'html.parser')

print(soup_c.select('div > p')[0])
print()

text_refine(str(soup_c.select('div > p')[0]))

<p><b>2016년 하계 올림픽 배구 남자 선수명단</b>은 2016년 8월6일부터 8월21일까지 <a>브라질</a> <a>리우데자네이루</a>에서 열리는 <a>제31회 하계 올림픽 남자 배구</a> 경기에 출전한 12개 남자 국가대표팀 선수 명단이다.
</p>



'2016년 하계 올림픽 배구 남자 선수명단은 2016년 8월6일부터 8월21일까지 브라질 리우데자네이루에서 열리는 제31회 하계 올림픽 남자 배구 경기에 출전한 12개 남자 국가대표팀 선수 명단이다.'

# 연습

In [ ]:
# 태그 삭제
text_raw = str(soup_c.select('p')[0]).replace('\n','')
text = ''
status = 0
for t in text_raw:
  if t == '<': status=1
  if status==0: text = text+t
  elif t == '>': status=0

text

'2016년 하계 올림픽 배구 남자 선수명단은 2016년 8월6일부터 8월21일까지 브라질 리우데자네이루에서 열리는 제31회 하계 올림픽 남자 배구 경기에 출전한 12개 남자 국가대표팀 선수 명단이다.'

In [ ]:
# <style> 태그 내용 제거

soup_t = BeautifulSoup(df['context'][995], 'html.parser')
temp = soup_t.select('div > p')[0]
# temp
# if '<style>' in temp:
#   idx_s = temp.find('<style>')
#   idx_f = temp.find('</style>')
#   print(idx_s, idx_f, temp[idx_s:idx_f+8])
#   deltxt = str(temp[idx_s:idx_f+8])
#   temp  = temp.replace(deltxt,'')

temp

<p><b>오쿠리가나</b>（<span><a>일본어</a>: </span><span><style>.mw-parser-output ruby>rt,.mw-parser-output ruby>rtc{font-feature-settings:"ruby"1}.mw-parser-output ruby.large{font-size:250%}.mw-parser-output ruby.large>rt,.mw-parser-output ruby.large>rtc{font-size:.3em}</style><span><ruby><rb>送</rb><rp> (</rp><rt></rt><rp>)</rp></ruby></span>りがな, <link/><span><ruby><rb>送</rb><rp> (</rp><rt>おく</rt><rp>)</rp></ruby></span>り<link/><span><ruby><rb>仮</rb><rp> (</rp><rt>が</rt><rp>)</rp></ruby></span><link/><span><ruby><rb>名</rb><rp> (</rp><rt>な</rt><rp>)</rp></ruby></span></span>)는 <a>일본어</a>를 <a>한자</a>와 <a>가나</a>를 섞어써서 표기할 때, 한자로 표기한 와고(<span><a>일본어</a>: </span><span>和語</span>, <link/><span><ruby><rb>倭語</rb><rp> (</rp><rt>わご</rt><rp>)</rp></ruby></span>), 또는 야마토코토바(<span><a>일본어</a>: </span><span>大和言葉</span>)를 쉽게 읽게 하기 위해 한자 뒤에 덧붙는 <a>가나</a>를 뜻한다. 이는 한 <a>단어</a> 내에서 한자를 쉽게 읽게 하기 위해 붙은 가나에 해당하고, 단어 자체가 가나만으로 이루어진 조사 같은 경우는 오쿠리가나라고 부르지 않는다. 또한, 오쿠리가나는 한자가 <a>훈독</a>으로 읽혔을 때만 사용되며, 음독으로 읽힐 때는 사용되지 않는다.
<

In [ ]:
print(temp)

<p><b>오쿠리가나</b>（<span><a>일본어</a>: </span><span><style>.mw-parser-output ruby>rt,.mw-parser-output ruby>rtc{font-feature-settings:"ruby"1}.mw-parser-output ruby.large{font-size:250%}.mw-parser-output ruby.large>rt,.mw-parser-output ruby.large>rtc{font-size:.3em}</style><span><ruby><rb>送</rb><rp> (</rp><rt></rt><rp>)</rp></ruby></span>りがな, <link/><span><ruby><rb>送</rb><rp> (</rp><rt>おく</rt><rp>)</rp></ruby></span>り<link/><span><ruby><rb>仮</rb><rp> (</rp><rt>が</rt><rp>)</rp></ruby></span><link/><span><ruby><rb>名</rb><rp> (</rp><rt>な</rt><rp>)</rp></ruby></span></span>)는 <a>일본어</a>를 <a>한자</a>와 <a>가나</a>를 섞어써서 표기할 때, 한자로 표기한 와고(<span><a>일본어</a>: </span><span>和語</span>, <link/><span><ruby><rb>倭語</rb><rp> (</rp><rt>わご</rt><rp>)</rp></ruby></span>), 또는 야마토코토바(<span><a>일본어</a>: </span><span>大和言葉</span>)를 쉽게 읽게 하기 위해 한자 뒤에 덧붙는 <a>가나</a>를 뜻한다. 이는 한 <a>단어</a> 내에서 한자를 쉽게 읽게 하기 위해 붙은 가나에 해당하고, 단어 자체가 가나만으로 이루어진 조사 같은 경우는 오쿠리가나라고 부르지 않는다. 또한, 오쿠리가나는 한자가 <a>훈독</a>으로 읽혔을 때만 사용되며, 음독으로 읽힐 때는 사용되지 않는다.
<

In [ ]:
temp.select('style')

[<style>.mw-parser-output ruby>rt,.mw-parser-output ruby>rtc{font-feature-settings:"ruby"1}.mw-parser-output ruby.large{font-size:250%}.mw-parser-output ruby.large>rt,.mw-parser-output ruby.large>rtc{font-size:.3em}</style>]

In [ ]:
# 특정 태그 제거
import re

re.sub('<style.*?>.*?</style>','',str(temp),re.I|re.S)

'<p><b>오쿠리가나</b>（<span><a>일본어</a>: </span><span><span><ruby><rb>送</rb><rp> (</rp><rt></rt><rp>)</rp></ruby></span>りがな, <link/><span><ruby><rb>送</rb><rp> (</rp><rt>おく</rt><rp>)</rp></ruby></span>り<link/><span><ruby><rb>仮</rb><rp> (</rp><rt>が</rt><rp>)</rp></ruby></span><link/><span><ruby><rb>名</rb><rp> (</rp><rt>な</rt><rp>)</rp></ruby></span></span>)는 <a>일본어</a>를 <a>한자</a>와 <a>가나</a>를 섞어써서 표기할 때, 한자로 표기한 와고(<span><a>일본어</a>: </span><span>和語</span>, <link/><span><ruby><rb>倭語</rb><rp> (</rp><rt>わご</rt><rp>)</rp></ruby></span>), 또는 야마토코토바(<span><a>일본어</a>: </span><span>大和言葉</span>)를 쉽게 읽게 하기 위해 한자 뒤에 덧붙는 <a>가나</a>를 뜻한다. 이는 한 <a>단어</a> 내에서 한자를 쉽게 읽게 하기 위해 붙은 가나에 해당하고, 단어 자체가 가나만으로 이루어진 조사 같은 경우는 오쿠리가나라고 부르지 않는다. 또한, 오쿠리가나는 한자가 <a>훈독</a>으로 읽혔을 때만 사용되며, 음독으로 읽힐 때는 사용되지 않는다.\n</p>'

In [ ]:
soup_c.select('div > p')[0].text

'오쿠리가나（일본어: .mw-parser-output ruby>rt,.mw-parser-output ruby>rtc{font-feature-settings:"ruby"1}.mw-parser-output ruby.large{font-size:250%}.mw-parser-output ruby.large>rt,.mw-parser-output ruby.large>rtc{font-size:.3em}送 ()りがな, 送 (おく)り仮 (が)名 (な))는 일본어를 한자와 가나를 섞어써서 표기할 때, 한자로 표기한 와고(일본어: 和語, 倭語 (わご)), 또는 야마토코토바(일본어: 大和言葉)를 쉽게 읽게 하기 위해 한자 뒤에 덧붙는 가나를 뜻한다. 이는 한 단어 내에서 한자를 쉽게 읽게 하기 위해 붙은 가나에 해당하고, 단어 자체가 가나만으로 이루어진 조사 같은 경우는 오쿠리가나라고 부르지 않는다. 또한, 오쿠리가나는 한자가 훈독으로 읽혔을 때만 사용되며, 음독으로 읽힐 때는 사용되지 않는다.\n'

In [ ]:
text_raw = soup_c.select('div > p')[0].text

del_li = ['\n','\xa0','[*]','[1]','[2]','[3]','[4]','[5]','[6]','[7]','[8]','[9]','\ufeff','()']

for d in del_li:
  if d in text_raw:
    idx_s = text_raw.find(d)
    ld = len(d)
    print(idx_s, text_raw[idx_s:idx_s+ld])
    deltxt = str(text_raw[idx_s:idx_s+ld])
    text = text_raw.replace(deltxt,'')

print(text_raw)
print(text)

494 

12 mw-parser-output
오쿠리가나（일본어: .mw-parser-output ruby>rt,.mw-parser-output ruby>rtc{font-feature-settings:"ruby"1}.mw-parser-output ruby.large{font-size:250%}.mw-parser-output ruby.large>rt,.mw-parser-output ruby.large>rtc{font-size:.3em}送 ()りがな, 送 (おく)り仮 (が)名 (な))는 일본어를 한자와 가나를 섞어써서 표기할 때, 한자로 표기한 와고(일본어: 和語, 倭語 (わご)), 또는 야마토코토바(일본어: 大和言葉)를 쉽게 읽게 하기 위해 한자 뒤에 덧붙는 가나를 뜻한다. 이는 한 단어 내에서 한자를 쉽게 읽게 하기 위해 붙은 가나에 해당하고, 단어 자체가 가나만으로 이루어진 조사 같은 경우는 오쿠리가나라고 부르지 않는다. 또한, 오쿠리가나는 한자가 훈독으로 읽혔을 때만 사용되며, 음독으로 읽힐 때는 사용되지 않는다.

오쿠리가나（일본어: . ruby>rt,. ruby>rtc{font-feature-settings:"ruby"1}. ruby.large{font-size:250%}. ruby.large>rt,. ruby.large>rtc{font-size:.3em}送 ()りがな, 送 (おく)り仮 (が)名 (な))는 일본어를 한자와 가나를 섞어써서 표기할 때, 한자로 표기한 와고(일본어: 和語, 倭語 (わご)), 또는 야마토코토바(일본어: 大和言葉)를 쉽게 읽게 하기 위해 한자 뒤에 덧붙는 가나를 뜻한다. 이는 한 단어 내에서 한자를 쉽게 읽게 하기 위해 붙은 가나에 해당하고, 단어 자체가 가나만으로 이루어진 조사 같은 경우는 오쿠리가나라고 부르지 않는다. 또한, 오쿠리가나는 한자가 훈독으로 읽혔을 때만 사용되며, 음독으로 읽힐 때는 사용되지 않는다.



In [ ]:
# 정규표현식 연습
import re

te = '빨강[1]((),) 주황[orange], (노랑[*], 초록[주 1], 파랑)\n, 남색\xa0, 보라[7]<style>djwjdklfjsioesijcvmkjdn 45 y436 dfs [1] [23ds] </style> 끝'
# 원하는 결과 : 빨강(,) 주황[orange], (노랑, 초록, 파랑), 남색, 보라 끝

text = re.sub('<style.*?>.*?</style>','',te,re.I|re.S)
text = re.sub('\n|\xa0|\ufeff|[[][*][]]','',text)
# text = re.sub('[(]+[)]','',text)
text = re.sub('[[]+\d+[]]','',text)
text = re.sub('[[주 ]+\d+[]]','',text)

if '()' in text:
  text = text.replace('()','')

# res1 = re.sub('[[]+\d+[]]','',te)
# res2 = del1.sub('',te)
# res2 = del2.sub('',te)

# print(res1)
# print(res2)
print(text)

빨강(,) 주황[orange], (노랑, 초록, 파랑), 남색, 보라 끝


In [ ]:
# 보관용 -- text 추출

check = [] ####
check2 = [] ####
text_li = []
for i in range(len(df['context'])):
  html = df['context'][i]
  soup = BeautifulSoup(html, 'html.parser')

  try:
    text_raw = str(soup.select('div > p')[0])
    text = text_refine(text_raw)

    if text[:3] == '좌표:':
      text_raw = str(soup.select('div > p')[1])
      text = text_refine(text_raw)

  except:
    text = 'No text'
  text_li.append(text)
  print(i, text) ####
  if text[:3] == '좌표:':####
    check.append((i,text))

    text_raw = str(soup.select('div > p')[1])
    text2 = text_refine(text_raw)
    check2.append((i,text2))

print(i, 'done')
# text_li

# 질의응답

In [ ]:
for i in range(10):
  print(df['qas'][i])

[{'answer': {'text': '<td>332cm</td>', 'html_answer_start': 45900, 'html_answer_text': '<td>332cm</td>', 'answer_start': 10698}, 'question': '파스칼레 소틸레의 스파이크 높이는 몇 cm인가?', 'id': '38824'}, {'answer': {'text': 'MKS 벵진', 'html_answer_start': 56978, 'html_answer_text': 'MKS 벵진', 'answer_start': 13498}, 'question': '2016년 하계 올림픽 캐나다 남자 배구 대표팀 선수들 중, 타일러 샌더스는 2015-16년에 어느 팀 소속이었나?', 'id': '11930'}]
[{'answer': {'text': '<td>145</td>', 'html_answer_start': 19379, 'html_answer_text': '<td>145</td>', 'answer_start': 3707}, 'question': '혈액 검사에서 나트륨의 정상범위는 최소 136mmol/L에서 최대 몇 mmol/L까지 일까?', 'id': '40175'}]
[{'answer': {'text': '사이프러스', 'html_answer_start': 29469, 'html_answer_text': '사이프러스', 'answer_start': 9036}, 'question': '안토닌 드보르자크의 작품 중 18개의 가곡으로 이루어진 노래는?', 'id': '26058'}]
[{'answer': {'text': '1695년', 'html_answer_start': 11274, 'html_answer_text': '1695년', 'answer_start': 1951}, 'question': '에드먼드 핼리가 달의 운동이 빨라지고 있다고 주장한 년도는 언제일까?', 'id': '58077'}, {'answer': {'text': '중력 토크', 'html_answer

In [ ]:
df['qas'][0]

[{'answer': {'answer_start': 10698,
   'html_answer_start': 45900,
   'html_answer_text': '<td>332cm</td>',
   'text': '<td>332cm</td>'},
  'id': '38824',
  'question': '파스칼레 소틸레의 스파이크 높이는 몇 cm인가?'},
 {'answer': {'answer_start': 13498,
   'html_answer_start': 56978,
   'html_answer_text': 'MKS 벵진',
   'text': 'MKS 벵진'},
  'id': '11930',
  'question': '2016년 하계 올림픽 캐나다 남자 배구 대표팀 선수들 중, 타일러 샌더스는 2015-16년에 어느 팀 소속이었나?'}]